### Importing the libraries :

In [ ]:
pip install -r requirements.txt

In [1]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from nltk.tokenize import word_tokenize
import joblib
import pandas as pd
from datetime import datetime

In [2]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

### Load the trained model

In [3]:
model = joblib.load('final_model.joblib')
vectorizer = joblib.load('final_vectorizer.joblib')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.2.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1

### Preprocess helper functions

In [4]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mukesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mukesh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### List of emotions from the Go Emotions dataset

In [5]:
emotions = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity',
    'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
    'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
    'remorse', 'sadness', 'surprise', 'neutral'
]

###  DataFrame for emotion lookup

In [6]:
emotion_df = pd.DataFrame({'emotion': emotions})

### Audit data storage

In [7]:
audit_data = []

In [8]:
def preprocess_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub('[^A-Za-z0-9 ]+', '', text)
    text = text.lower()
    text = re.sub('\d+', '', text)
    text = word_tokenize(text)
    text = [word for word in text if word not in stop_words]
    text = [stemmer.stem(word) for word in text]
    text = ' '.join(text)
    return text

In [9]:
@app.route('/')
def home():
    return render_template('home.html')

In [10]:
@app.route('/predict', methods=['POST'])
def predict():
    # Get the text input from the request
    if request.is_json:
        data = request.get_json()
        text = data.get('text')
    else:
        # Get the text input from form data
        text = request.form.get('text')

    if text is None:
        return jsonify({'error': 'Invalid input'})

    # Preprocess the text input
    text = preprocess_text(text)

    # Transform the preprocessed text using the vectorizer
    text_vectorized = vectorizer.transform([text])

    # Make predictions using the loaded model
    predicted_class = model.predict(text_vectorized)[0]

    # Retrieve the emotion based on the predicted class index
    emotion = get_emotion(predicted_class)

    # Store input, result, date, and time in the audit data
    now = datetime.now()
    audit_data.append({'input': request.form.get('text'), 'class': predicted_class, 'emotion': emotion, 'date': now.date(), 'time': now.time()})

    results = {
        'input_text': request.form.get('text'),
        'predicted_class': predicted_class,
        'emotion': emotion
    }

    
    return render_template('results.html', input_text=request.form.get('text'), predicted_class=predicted_class, emotion=emotion)

In [11]:
@app.route('/audit')
def audit():
    return render_template('audit.html', audit_data=audit_data)

In [12]:
def get_emotion(predicted_class):
    if predicted_class < 0 or predicted_class >= len(emotions):
        return 'Unknown'
    return emotion_df.loc[predicted_class, 'emotion']

In [13]:
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/traitlets/config/application.py", line 1040, in launch_instance
    app.initialize(argv)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/

SystemExit: 1

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
